In [ ]:
import subprocess
import schedule
import time
from datetime import datetime

def send_imessage(contact, message):
    """
    Sends an iMessage using AppleScript, targeting the first 
    iMessage account on your Mac.
    """
    applescript = f'''
    tell application "Messages"
        set targetService to 1st account whose service type = iMessage
        send "{message}" to buddy "{contact}" of targetService
    end tell
    '''
    # Run AppleScript; 'check=True' throws error if AppleScript fails
    subprocess.run(["osascript", "-e", applescript], check=True)

def scheduled_job(contact, message):
    """
    Sends the message and logs a timestamp.
    """
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"[{current_time}] Sending iMessage to: {contact}")
    try:
        send_imessage(contact, message)
        print("Message sent successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Failed to send iMessage: {e}")

# ---------------- MAIN SCRIPT ---------------- #

# 1. Prompt the user for a date
input_date = input("Enter a date (YYYY-MM-DD): ")

# 2. Validate the date format
try:
    user_date = datetime.strptime(input_date, "%Y-%m-%d").date()
except ValueError:
    print("Invalid date format. Please enter the date as YYYY-MM-DD.")
    exit(1)

# 3. Define your iMessage details
contact = "+11234567890"  # <--- Replace with your own phone number or iMessage email
message = "Message + Link(https://open.spotify.com/track/4PTG3Z6ehGkBFwjybzWkR8?si=efb9a8e41d9a4e68)"

# 4. Immediately send an iMessage
now_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f"[{now_str}] Sending immediate iMessage (date entered: {user_date})...")

try:
    send_imessage(contact, message)
    print("Immediate message sent successfully!")
except subprocess.CalledProcessError as e:
    print(f"Failed to send immediate iMessage: {e}")

# 5. Record the current local time (HH:MM) for daily scheduling
time_for_daily = datetime.now().strftime('%H:%M')  # e.g. "15:45"

# Schedule to repeat daily at the exact same time
schedule.every().day.at(time_for_daily).do(scheduled_job, contact, message)

print(f"All future messages will be sent daily at {time_for_daily}.")
print("Scheduler is running. Press Ctrl+C to exit.")

# 6. Keep the script running, checking for scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(60)  # check the schedule every 60 seconds